ユーザ素性ベクトルとアイテム素性ベクトルが与えられた場合の一般的なスコアリングアプローチの１つは、<strong>それらの素性ベクトルを用いてユーザ$i$とアイテム$j$の間の親和性を測るスコアリング関数$s\left( {\vec {x}}_{i}, {\vec {x}}_{j} \right)$を設計することである</strong>。そして、スコアが求まれば、それに基づいてアイテムを順位付けすることで推薦を行うことができる。

In [1]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# 日本語フォントを設定
font = {'family': 'IPAexGothic'}
mpl.rc('font', **font)

%matplotlib inline

In [2]:
def read_dataset_series():
    return pd.read_csv(
        'https://s3-ap-northeast-1.amazonaws.com/wp.lancers.jp/engineerblog/wp-content/uploads/2017/11/lancers_recommend_datasets.json_-1.zip',
        compression='zip'
    ).T.index.str.extract('\{*\"[0-9]+\": \"(.+)\"\}*')[0]

In [3]:
文書_ser = read_dataset_series()
文書_ser.head(2)

0    中国語 コミュニティサイト 翻訳 仕事 中国 コミュニティサイト 翻訳 募集 個人的 研究 ...
1    現行 Blu-ray Discレコーダー 商品 データ 作成 現状 販売 Blu-ray D...
Name: 0, dtype: object

In [4]:
文書_ser.tail(2)

9998    テーマ 修正 テーマ 修正 テーマ 下記 修正 お願い テーマ 設定 完了 済 修正 テーマ...
9999    blog 記事 更新 自動 ツール 羅列 記事 紹介 サイト 羅列 記事 作成 アクセス ラ...
Name: 0, dtype: object

In [5]:
class DocumentSeries(pd.Series):
    def __init__(self, document : pd.Series):
        super().__init__(document)

In [6]:
document_ser = DocumentSeries(文書_ser)

In [7]:
class VectorDataFrame(pd.DataFrame):
    def __init__(self, vector_arr: np.array):
        super().__init__(vector_arr)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
vector_arr = tfidf.fit_transform(CountVectorizer().fit_transform(document_ser)).toarray()

In [9]:
vector_df = VectorDataFrame(vector_arr)
vector_df.head()

,0,1,2,3,4,5,6,7,8,9,...,19125,19126,19127,19128,19129,19130,19131,19132,19133,19134
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 教師なし手法

## 距離関数用いたスコアリング関数

### ユークリッド距離

$$
d\left( \vec {x}_{i}, \vec {x}_{j} \right) = \sqrt { \left( x_{i1} - x_{j1} \right)^{2} + \left( x_{i2} - x_{j2} \right)^{2} + \cdots + \left( x_{ip} - x_{jp} \right)^{2} }
$$

In [11]:
from sklearn.metrics.pairwise import euclidean_distances

euclidean_distances(vector_df)

array([[0.        , 1.4021506 , 1.39029598, ..., 1.4070818 , 1.41421356,
        1.39809281],
       [1.4021506 , 0.        , 1.39357315, ..., 1.39149314, 1.40804872,
        1.39244078],
       [1.39029598, 1.39357315, 0.        , ..., 1.4029932 , 1.41234596,
        1.39527745],
       ...,
       [1.4070818 , 1.39149314, 1.4029932 , ..., 0.        , 1.39714695,
        1.41421356],
       [1.41421356, 1.40804872, 1.41234596, ..., 1.39714695, 0.        ,
        1.41421356],
       [1.39809281, 1.39244078, 1.39527745, ..., 1.41421356, 1.41421356,
        0.        ]])

### マハラノビス距離

## 類似度関数用いたスコアリング関数

### コサイン類似度
$\vec {x}_{i}$と$\vec {x}_{j}$が同じベクトル空間内の点であるとする。
$$
s\left( {\vec {x}}_{i}, {\vec {x}}_{j} \right) = \frac { {\vec {x}}^{\prime}_{i} {\vec {x}}_{j} }{ \left\| \vec {x}_{i} \right\| \cdot \left\| \vec {x}_{j} \right\| }
$$

 - $\vec {x}_{i}$ : ユーザ$i$の素性ベクトル
 - $\vec {x}_{j}$ : アイテム$j$の素性ベクトル

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(vector_df)

array([[1.        , 0.01698685, 0.03353854, ..., 0.0100604 , 0.        ,
        0.02266824],
       [0.01698685, 1.        , 0.02897694, ..., 0.03187342, 0.0086994 ,
        0.03055434],
       [0.03353854, 0.02897694, 1.        , ..., 0.01580504, 0.00263944,
        0.02660041],
       ...,
       [0.0100604 , 0.03187342, 0.01580504, ..., 1.        , 0.0239902 ,
        0.        ],
       [0.        , 0.0086994 , 0.00263944, ..., 0.0239902 , 1.        ,
        0.        ],
       [0.02266824, 0.03055434, 0.02660041, ..., 0.        , 0.        ,
        1.        ]])

### ピアソンの相関係数

### 偏差パターン類似度

## 集合の類似度

### ジャッカード係数

In [ ]:
# from sklearn.metrics.pairwise import pairwise_distances

# pairwise_distances(vector_df, metric='jaccard')

### ダイアス係数

### シンプソン係数

---

## 教師あり手法
教師あり手法では、過去のユーザ-アイテム間の相互作用によって収集された観測済みの<strong style="color:blue;">レイティングデータ</strong>を教師データとし、それぞれの素性ベクトルを用いて、観測されていないユーザ-アイテム対のレイティングを予測するモデルを学習する。

$$
\begin{eqnarray}
s_{ij} = s\left( \vec {x}_{i}, \vec {x}_{j} \right) & = & {\vec {x}}^{\prime}_{i} {\bf A} \vec {x}_{j}\\
& = & \left( \begin{matrix} { x }_{ i1 } & { x }_{ i2 } & { x }_{ i3 } \end{matrix} \right) \left( \begin{matrix} { a }_{ 11 } & { a }_{ 12 } \\ { a }_{ 21 } & { a }_{ 22 } \\ { a }_{ 31 } & { a }_{ 32 } \end{matrix} \right) \left( \begin{matrix} { x }_{ j1 } \\ { x }_{ j2 } \end{matrix} \right) \\
& = & {\vec {x}}^{\prime}_{ij} \vec {\beta}
\end{eqnarray}
$$

 - $s_{ij}$ : ユーザ$i$のアイテム$j$に対するスコア
 - $\vec {x}_{i}$ : ユーザ$i$の素性ベクトル
 - $\vec {x}_{j}$ : アイテム$j$の素性ベクトル
 - ${\vec {x}}^{\prime}_{ij} = \left( \begin{matrix} { x }_{ i1 }{ x }_{ j1 } \\ { x }_{ i2 }{ x }_{ j1 } \\ { x }_{ i3 }{ x }_{ j1 } \\ { x }_{ i1 }{ x }_{ j2 } \\ { x }_{ i2 }{ x }_{ j2 } \\ { x }_{ i3 }{ x }_{ j2 } \end{matrix} \right) $
 - $\vec {\beta} = \left( \begin{matrix} { a }_{ 11 } \\ { a }_{ 21 } \\ { a }_{ 31 } \\ { a }_{ 12 } \\ { a }_{ 22 } \\ { a }_{ 32 } \end{matrix} \right) $

### バイナリレイティング(ロジスティックモデル)

ユーザ$i$がアイテム$j$に与えるレイティング$y_{ij} \in \left\{ +1, -1 \right\}$(例えば、ユーザ$i$がアイテム$j$をクリックするか否か)が、以下のロジスティック応答モデルに基づいて生成されると仮定する。

$$
y_{ij} \sim \mathrm {Bernoulli}\left( \vec {x}_{i}, \vec {x}_{j} \right) = \frac {1}{1 + \mathrm {exp}\left\{ - s_{ij} \right\}}
$$

観測された(ユーザ$i$,アイテム$j$)対の集合を$\Omega$とし、観測されたレイティングを${\bf Y} = \left\{ y_{ij} : \left(i,j\right) \in \Omega \right\}$とする。

パラメータ${\bf A}$の推定方法は、対数尤度関数より求められる。

 - [machine-learning/ロジスティック回帰.ipynb at master · gtaiyou24/machine-learning](https://github.com/gtaiyou24/machine-learning/blob/master/%E5%88%86%E9%A1%9E/%E3%83%AD%E3%82%B8%E3%82%B9%E3%83%86%E3%82%A3%E3%83%83%E3%82%AF%E5%9B%9E%E5%B8%B0.ipynb)

### 数値型レイティング(ガウシアンモデル)

ユーザ$i$がアイテム$j$に与える数値型レイティング$y_{ij}$(例えば、ユーザ$i$がアイテム$j$に与える点数または星の数)は、以下のガウス応答モデルに基づいて生成されると仮定する。

$$
y_{ij} \sim N\left( s_{ij}, \sigma^{2} \right)
$$

 - [machine-learning/線形回帰モデル.ipynb at master · gtaiyou24/machine-learning](https://github.com/gtaiyou24/machine-learning/blob/master/%E5%9B%9E%E5%B8%B0%E5%88%86%E6%9E%90/%E7%B7%9A%E5%BD%A2%E5%9B%9E%E5%B8%B0%E3%83%A2%E3%83%87%E3%83%AB.ipynb)

### 順序レイティング(累積ロジットモデル)

> 多くのアプリケーションでは、レイティングは本質的に$k$段階スケールで表される。ガウシアンモデルは順序レイティング(例えば、星の数)で一般的に使用されるが、この方法は最良ではないかもしれない。例えば、星５つと星４つの格付けの違いは、星３つと星４つの格付けの違いと同じではないこともある。

この場合、順序回帰がより適切だと考えられる。このモデルでは、ユーザ$i$がアイテム$j$に対して与える評価$y_{ij} \in \left\{ 1, \dots, R \right\}$は多項分布に従って生成されると仮定している。

$$
y_{ij} \sim \mathrm {Multinomial}\left( \pi_{ij,1}, \dots, \pi_{ij,R} \right)
$$

ここで、$\pi_{ij,r}$は、ユーザ$i$がアイテム$j$にレイティング$r$を与える確率である。$Y_{ij}$を観測されたレイティング$y_{ij}$に対応する確率変数とする。

### 共起嗜好性スコア

> 一部のアプリケーションの設定では、ユーザが１つのアイテムを別のアイテムよりも好む傾向を観察できる。あるいは、ユーザの反応を共起嗜好性に変換することができる(例えば、同一ユーザがクリックしたアイテムは、クリックされていないアイテムよりも優れていると考える)。

このような嗜好データをモデル化するには、記法を少し変更しなければならない。

$y_{ij\ell} \in \left\{ +1, -1 \right\}$は、ユーザ$i$がアイテム$\ell$に対してアイテム$j$をより好むかどうかを示すものとする。$\Omega$を観測された$\left(i,j,\ell\right)$の３組の集合とする。ユーザ$i$がアイテム$\ell$よりもアイテム$j$を好む傾向は、$s_{ij} - s_{i\ell}$に比例すると仮定する。つまり、

$$
y_{ij\ell} \sim \mathrm {Bernoulli}\left( \vec {x}_{i}, \vec {x}_{j}, \vec {x}_{\ell} \right) = \frac {1}{1 + \mathrm {exp}\left\{ - \left(s_{ij} - s_{i\ell}\right) \right\}}
$$

とする。

### 正則化付き最尤推定
正則化項を追加すると過学習が軽減されますよーという話

---

# コンテキスト情報

これまでは、ユーザ$i$がアイテム$j$に与えるレイティング$y_{ij}$を予測する方法のみを検討してきた。しかし、そのようなレイティングは通常コンテキストに依存する。

> 例えば、$y_{ij}$(ユーザ$i$がアイテム$j$をクリックするかどうか)は、Webページ上のアイテムの位置に依存すると考えられる。目を引く位置にある場合、同じアイテムでも、それほど目立たない位置にある場合よりも高いクリック率が得られる。クリック率は、時刻/平日と週末/デバイス(デスクトップPCとモバイル)/同じWebページに表示されるその他のアイテムなどによっても変わる可能性がある。

このようなコンテキスト情報を素性ベクトル$\vec {z}_{ij}$として表す。このベクトルはユーザ$i$がアイテム$j$との相互作用をする際のコンテキストを特徴付ける。教師あり手法は推定レイティングまたはスコアを次のように再定義することによって、容易にコンテキスト情報を取り込むことができる。

$$
s_{ij} = s\left( \vec {x}_{i}, \vec {x}_{j}, \vec {z}_{ij} \right) = {\vec {x}}^{\prime}_{i} {\bf A} \vec {x}_{j} + \vec {b}^{\prime} \vec {z}_{ij}
$$